In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import Delaunay
from sklearn.metrics.cluster import fowlkes_mallows_score
import vg
import plotly.graph_objects as go

from pytransform3d.rotations import matrix_from_axis_angle
from scipy.spatial.transform import Rotation as Rot
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

cube_coor=(np.transpose(np.array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)*0.1

def _rotmat( vector, points):
    """
    Rotates a 3xn array of 3D coordinates from the +z normal to an
    arbitrary new normal vector.
    """
    
    vector = vg.normalize(vector)
    axis = vg.perpendicular(vg.basis.z, vector)
    angle = vg.angle(vg.basis.z, vector, units='rad')
    
    a = np.hstack((axis, (angle,)))
    R = matrix_from_axis_angle(a)
    
    r = Rot.from_matrix(R)
    rotmat = r.apply(points)
    
    return rotmat

def mesh_cube(coor,v):
    
    
    rot_cube=_rotmat(v,coor)
    return go.Mesh3d(
    x=rot_cube[:,0]+v[0],
    y=rot_cube[:,1]+v[1],
    z=rot_cube[:,2]+v[2],
    opacity=0.3,
    i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
    j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
    k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],color='cyan'
    )

def go_cluster(cluster):
    
    if(len(cluster.galaxies)==1):
        return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=1,
        opacity=1,
        color='green'
    )
    )]
    
    return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=1,
        opacity=1,
        color='green'

    )
    )]+[cluster.get_mesh()]+[go.Scatter3d(
    x=[cluster.centroid[0]],
    y=[cluster.centroid[1]],
    z=[cluster.centroid[2]],
    mode='markers',
    marker=dict(
        size=1,
        opacity=1,
        color='red'
    )
    )]

def plot_clusters(clusters):
    
    data=[]
    for cluster in clusters:
        data+=go_cluster(cluster)
    
    fig = go.Figure(data=data)

# tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [2]:
coor=pd.read_csv('ra_dec_z_data.csv')

In [3]:

tmp_coor=coor[['x','y','z']]

tmp_coor=tmp_coor[tmp_coor.x.between(20, 40)]
tmp_coor=tmp_coor[tmp_coor.y.between(-10, 10)]
tmp_coor=tmp_coor[tmp_coor.z.between(-10, 10)]



In [22]:
class Cluster():
    # non shifted shape, easy to expand
    non_rotated_cube=(np.transpose(np.array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)*0.3
    rotated_cube=None# shifted coor of rotated cube
    centroid=None# coor of center
    galaxies=None

    def __init__(self, center,non_rotated_cube=None,galaxies=None):
        
        if(isinstance(non_rotated_cube, np.ndarray)):
            self.non_rotated_cube=non_rotated_cube
        if(not isinstance(galaxies, np.ndarray)):
            self.galaxies=np.array([center])
        else:
            self.galaxies=np.array(galaxies)
        
        self.centroid=center
        self.rotated_cube = self.rotate(self.centroid,self.non_rotated_cube)+center
        
        for gal in self.galaxies:
            if(Delaunay(self.rotated_cube).find_simplex(gal) < 0):
                raise ValueError
        
        
    def rotate(self, vector, points):    
        vector = vg.normalize(vector)
        axis = vg.perpendicular(vg.basis.z, vector)
        angle = vg.angle(vg.basis.z, vector, units='rad')
        
        a = np.hstack((axis, (angle,)))
        R = matrix_from_axis_angle(a)
        
        r = Rot.from_matrix(R)
        rotmat = r.apply(points)
        
        return rotmat

    def get_mesh(self):
        return go.Mesh3d(
        x=self.rotated_cube[:,0],
        y=self.rotated_cube[:,1],
        z=self.rotated_cube[:,2],
        opacity=0.5,
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],color='blue'
        )
    
    
class Clusterer:
    
    unit_cube=(np.transpose(np.array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)

    clusters=[]
        
    def __init__(self, data):
        
        for coor in np.array(data):
            self.clusters.append(Cluster(coor))
    
    def merge(self):
        
        clusters_len=len(self.clusters)
        graph=np.zeros((clusters_len,clusters_len))
        for i in tqdm(range(clusters_len)):
            for j in range(clusters_len):
                if(np.linalg.norm(self.clusters[i].centroid-self.clusters[j].centroid)>3):
                    continue
                if(self.check_collision(self.clusters[i],self.clusters[j])):
                    graph[i,j]=1
                    
        graph = csr_matrix(graph)
        n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
        
        components=[]
        
        np_clusters=np.array(self.clusters)
        
        for label in np.unique(labels):
            components.append(np_clusters[labels==label])
        
        new_clusters=[]
 
        for component in components:
            if(len(component)==1):
                new_clusters.append(component[0])
            else:
                new_clusters.append(self.collide_clusters(component))
        
        self.clusters=new_clusters
        
        
    def check_collision(self,a:Cluster, b: Cluster):        
        for point in b.rotated_cube:
            if(Delaunay(a.rotated_cube).find_simplex(point) >= 0):
                return True   
        return False
    
    def rotation_matrix_from_vectors(self, vec2):
        vec1=[1,0,0]
        a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
        v = np.cross(a, b)
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
        return rotation_matrix
    
    def collide_clusters(self, clusters:[Cluster]):
        
        galaxies=[]
        vertex_points=[]
        for cluster in clusters:
            galaxies.append(cluster.galaxies)
            vertex_points.append(cluster.rotated_cube)
        galaxies=np.concatenate( galaxies, axis=0 )
        vertex_points=np.concatenate( vertex_points, axis=0 )
        
        center = galaxies.mean(axis=0)

        projections = center * np.dot(galaxies, np.transpose([center])) / np.dot(center, center)
        
        distances_on_line=np.linalg.norm(projections-center,axis=1)
        vectors_from_line=galaxies-projections
    
        length = distances_on_line.max()
        width=np.linalg.norm(vectors_from_line,axis=1).max()
    
        cube=self.unit_cube.copy()*[width*2+0.05,width*2+0.05,length*2+0.05]
    
        return Cluster(center,cube,galaxies)
            

In [23]:
clusterer=Clusterer(tmp_coor)
clusterer.merge()


100%|████████████████████████████████████████████████████████████████████████████████| 341/341 [00:33<00:00, 10.20it/s]


In [24]:
plot_clusters(clusterer.clusters)